### Classification Decision Tree

# Classification with hyperparameter tuning

### aim:
Show classification with different strategies for the tuning and evaluation of the classifier
1. simple __holdout__
2. __holdout with validation__ train and validate repeatedly changing a hyperparameter, to find the value giving the best score, then test for the final score
4. __cross validation__ on training set, then score on test set

    

### Workflow
- download the data
- drop the useless data
- separe the predicting attributes X from the class attribute y
- split X and y into training and test

- part 1 - single run with default parameters
    - initialise an estimator with the chosen model generator
    - fit the estimator with the training part of X
    - show the tree structure
    - part 1.1 
        - predict the y values with the fitted estimator and the train data
            - compare the predicted values with the true ones and compute the accuracy on the training set 
    - part 1.2
        - predict the y values with the fitted estimator and the test data
            - compare the predicted values with the true ones and compute the accuracy on the test set 

- part 2 - multiple runs changing a parameter
    - prepare the structure to hold the accuracy data for the multiple runs
    - repeat for all the values of the parameter
        - initialise an estimator with the current parameter value
        - fit the estimator with the training part
        - predict the class for the test part
        - compute the accuracy and store the value
    - find the parameter value for the top accuracy

- part 3 - compute accuracy with cross validation
    - prepare the structure to hold the accuracy data for the multiple runs
    - repeat for all the values of the parameter
        - initialise an estimator with the current parameter value
        - compute the accuracy with cross validation and store the value
    - find the parameter value for the top accuracy
    - fit the estimator with the entire X
    - show the resulting tree and classification report

The data are already in your folder

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score


%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 20]
random_state = 14
np.random.seed(random_state)
# the random state is reset here in numpy, all the scikit-learn procedure use the numpy random state
# obviously the experiment can be repeated exactly only with a complete run of the program


data_url = "winequality-red.csv"
target_name = 'quality'
to_drop = []
# parameter_values to be determined after the fitting of the full tree
df = pd.read_csv(data_url , sep = ';')
print("Shape of the input data {}".format(df.shape))

Shape of the input data (1599, 12)


Have a quick look to the data.
- use the .shape attribute to see the size
- use the `.head()` function to see column names and some data
- use the `.hist()` method for an histogram of the columns
- use the .unique method to see the class values

Use the `hist` method of the DataFrame to show the histograms of the attributes

NB: a semicolon at the end of a statement suppresses the `Out[]`

Print the unique class labels (hint: use the `unique` method of pandas Series

#### Split the data into the predicting values X and the class y
Drop also the columns which are not relevant for training a classifier, if any

The method "drop" of dataframes allows to drop either rows or columns
- the "axis" parameter chooses between dropping rows (axis=0) or columns (axis=1)

Another quick look to data

## Prepare a simple model selection: holdout method
- Split X and y in train and test
- Show the number of samples in train and test, show the number of features

## Part 1

- Initialize an estimator with the required model generator `tree.DecisionTreeClassifier(criterion="entropy")`
- Fit the estimator on the train data and target

Look at the tree structure
- the feature names are used to show the tests in the nodes
    - they are the column names in the X
- the class names 
    - the attribute `estimator.classes_` contains the array of classes detected in the target; if the classes are numbers they have to be transformed in strings with `str()`
- the dept of the visualization can be limited with the parameter `max_depth` 
    
`plt.figure(figsize = (20,20))
 tree.plot_tree(estimator
          , filled=True
          , feature_names = X.columns
          , class_names = str(estimator.classes_)
          , rounded = True
          , proportion = True
          , max_depth = 1
              );`

### Part 1.1

Let's see how it works on training data
- predict the target using the fitted estimator on the training data
- compute the accuracy on the training set using `accuracy_score(<target>,<predicted_target) * 100`

### Part 1.2

That's more significant: how it works on test data
- use the fitted estimator to predict using the test features
- compute the accuracy and store it on a variable for the final summary
- store the maximum depth of the tree, for later use 
    - `fitted_max_depth = estimator.tree_.max_depth`
- store the range of the parameter which will be used for tuning
    - `parameter_values = range(1,fitted_max_depth+1)`
- print the accuracy on the test set and the maximum depth of the tree

## Part 2

Optimising the tree: limit the maximum tree depth. We will use the three way splitting: `train, validation, test`. For simplicity, since we already splitted in _train_ and _test_, we will furtherly split the _train_
- split the training set into two parts: __train_t__ and __val__
- max_depth - pruning the tree cutting the branches which exceed max_depth
- the experiment is repeated varying the parameter from 1 to the depth of the unpruned tree
- the scores for the various values are collected and plotted

### Loop for computing the score varying the hyperparameter
- initialise a list to contain the scores
- loop varying `par` in `parameter_values`
    - initialize an estimator with a DecisionTreeClassifier, using `par` as maximum depth and `entropy` as criterion
    - fit the estimator on the `train_t` part of the features and the target 
    - predict with the estimator using the validation features
    - compute the score comparing the prediction with the validation target and append it to the end of the list

### Plot the results
Plot using the `parameter_values` and the list of `scores`

In [18]:
#plt.figure(figsize=(32,20))
#plt.plot(parameter_values, scores, '-o', linewidth=5, markersize=24)
#plt.xlabel('max_depth')
#plt.ylabel('accuracy')
#plt.title("Score with validation varying max_depth of tree", fontsize = 24)
#plt.show();

### Fit the tree after validation and print summary
- store the parameter value giving the best score with `np.argmax(scores)`
- initialize an estimator as a DecisionTreeClassifier, using the best parameter value computed above as maximum depth and `entropy` as criterion
- fit the estimator using the `train` part
- use the fitted estimator to predict using the test features
- compute the accuracy on the test and store it on a variable for the final summary
- print the accuracy on the test set and the best parameter value

## Part 3 - Tuning with __Cross Validation__
Optimisation of the hyperparameter with __cross validation__ (cv suffix in the variable names).
Now we will tune the hyperparameter looping on cross validation with the __training set__, then we will fit the estimator on the training set and evaluate the performance on the __test set__

- initialize an empty list for the scores
- loop varying `par` in `parameter_values`
    - initialize an estimator with a DecisionTreeClassifier, using `par` as maximum depth and `entropy` as criterion
    - compute the score using the estimator on the `train` part of the features and the target using 
        - `cross_val_score(estimator, X_train, y_train, scoring='accuracy', cv = 5)`
        - the result is list of scores
    - compute the average of the scores and append it to the end of the list
- print the scores

Plot using the `parameter_values` and the list of `scores`

In [21]:
#plt.figure(figsize=(32,20))
#plt.plot(parameter_values,scores,'-o',linewidth=5,markersize=25)
#plt.xlabel('max_depth')
#plt.ylabel('accuracy')
#plt.title("Score with validation varying max_depth of tree", fontsize = 24)
#plt.show();

### Fit the tree after cross validation and print summary
- store the parameter value giving the best score with `np.argmax(scores)`
- initialize an estimator as a DecisionTreeClassifier, using the best parameter value computed above as maximum depth and `entropy` as criterion
- fit the estimator using the `train` part
- use the fitted estimator to predict using the test features
- compute the accuracy on the test and store it on a variable for the final summary
- print the accuracy on the test set and the best parameter value

- **micro**:
Calculate metrics globally by counting the total true positives, false negatives and false positives.
- **macro**:
Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.
- **weighted**:
Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.


Print a summary of the four experiments

In [ ]:
Observations made from the above four experiments:
1) By looking at the difference between the accuracies of train at first  and test data later
2) accuracy of train data reaches to almost 100% which represents the no bias
3) accuracy of test data also varies a lot of difference from the accurancy of train data which represents the high variance
4) In machine learning language , a model which is in above two points situation means, model is overfitted


### Suggested exercises
- try other datasets
- try to optimise the parameters "min_impurity_decrease" "min_samples_leaf" and "min_samples_split"